In [34]:
from __future__ import annotations

class Fruit:
    def __init__(self, name, color) -> None:
        self.name = name
        self.color = color

class Apple:
    def __init__(self, color, weight, price, origin):
        self.color = color
        self.weight = weight
        self.price = price
        self.origin = origin
    
    def get_fruit(self) -> Fruit:
        return Fruit("apple", self.color)
    


In [35]:
from typing import Any

def get_type_annotation(data: Any) -> str:
    # Check if the input is a dictionary
    if isinstance(data, dict):
        # Get the class name of the first non-None value in the dictionary
        class_name = next((value.__class__.__name__ for value in data.values() if value is not None), None)

        if class_name:
            return f"dict[str, {class_name}]"
        else:
            return "dict[str, Any]"
    # If the input is not a dictionary, return its type
    else:
        return type(data).__name__

In [36]:
def get_object_string(obj):
    if isinstance(obj, str):
        return f'"{obj}"'
    elif isinstance(obj, (int, float, bool)):
        return str(obj)
    elif isinstance(obj, list):
        return '[' + ', '.join(get_object_string(item) for item in obj) + ']'
    elif isinstance(obj, tuple):
        return '(' + ', '.join(get_object_string(item) for item in obj) + ')'
    elif isinstance(obj, dict):
        return '{' + ', '.join(f"{get_object_string(key)}: {get_object_string(value)}" for key, value in obj.items()) + '}'
    elif hasattr(obj, '__dict__'):
        args = ', '.join(f"{key}={get_object_string(value)}" for key, value in vars(obj).items())
        return f"{obj.__class__.__name__}({args})"
    else:
        return str(obj)

In [54]:
apple1 = Apple("red", 100, 100, "USA")
a = {
    "a1": apple1,
    "a2": Apple("green", 200, 200, "Japan"),
    "a3": Apple("yellow", 300, 300, "China"),
}

In [47]:
get_type_annotation(a)

'dict[str, Apple]'

In [48]:
get_object_string(a)

'{"a1": Apple(color="red", weight=100, price=100, origin="USA"), "a2": Apple(color="green", weight=200, price=200, origin="Japan"), "a3": Apple(color="yellow", weight=300, price=300, origin="China")}'

In [57]:
a = Apple("red", 100, 10, "China")

In [50]:
get_type_annotation(a)

'Apple'

In [51]:
get_object_string(a)

'Apple(color="red", weight=100, price=10, origin="China")'

In [52]:
a = 10
print(get_type_annotation(a), get_object_string(a))

int 10


In [58]:
print(f"'Apple Examples' (a) ({get_type_annotation(a)}) = {get_object_string(a)}")

'Apple Examples' (a) (Apple) = Apple(color="red", weight=100, price=10, origin="China")
